# Play QWOP Game with RL

## CS496 madcamp week4 qwop

In [3]:
import read_qwop_game as qwop

In [16]:
import numpy as np
from PIL import Image
import cv2
import io
import time
import pandas as pd
import numpy as np
from IPython.display import clear_output
from random import randint
import os

from keras.models import model_from_json
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD , Adam
from keras.callbacks import TensorBoard
from collections import deque
import random
import pickle
from io import BytesIO
import base64
import json
import queue
import threading

In [17]:
loss_file_path = "./objects/loss_df.csv"
# actions_file_path = "./objects/actions_df.csv"
q_value_file_path = "./objects/q_vales.csv"
scores_file_path = "./objects/scores_df.csv"

In [18]:
# game parameters
ACTIONS = 4 # possible actions: q, w, o, p
GAMMA = 0.99 # decay rate of past observations original 0.99
OBSERVATION = 100. # timesteps to observe before training
EXPLORE = 60000  # frames over which to anneal epsilon
FINAL_EPSILON = 0.0001 # final value of epsilon
INITIAL_EPSILON = 0.1 # starting value of epsilon
REPLAY_MEMORY = 50000 # number of previous transitions to remember
BATCH = 16 # size of minibatch
FRAME_PER_ACTION = 1
LEARNING_RATE = 1e-04
img_rows, img_cols = 80, 80
img_channels = 4 # We stack 4 frames

In [19]:
# Intialize log structures from file if exists else create new
loss_df = pd.read_csv(loss_file_path) if os.path.isfile(loss_file_path) else pd.DataFrame(columns =['loss'])
scores_df = pd.read_csv(scores_file_path) if os.path.isfile(loss_file_path) else pd.DataFrame(columns = ['scores'])
# actions_df = pd.read_csv(actions_file_path) if os.path.isfile(actions_file_path) else pd.DataFrame(columns = ['actions'])
q_values_df =pd.read_csv(actions_file_path) if os.path.isfile(q_value_file_path) else pd.DataFrame(columns = ['qvalues'])

NameError: name 'actions_file_path' is not defined

In [8]:
def save_obj(obj, name):
    with open('objects/' + name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
def load_obj(name):
    with open('objects/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

# Show images in new window
def show_img(graphs = False):
    while True:
        screen = (yield)
        window_title = "logs" if graphs else "game_play"
        cv2.namedWindow(window_title, cv2.WINDOW_NORMAL)
        cv2.resizeWindow(window_title, 300,300)
        cv2.imshow(window_title, screen)
        if (cv2.waitKey(1) & 0xFF == ord('q')):
            cv2.destroyAllWindows()
            break

In [9]:
# training variables saved as checkpoints to filesystem to resume training from the same step
def init_cache():
    """initial variable caching, done only once"""
    save_obj(INITIAL_EPSILON,"epsilon")
    t = 0
    save_obj(t,"time")
    D = deque()
    save_obj(D,"D")

In [10]:
'''Call only once to init file structure
'''
#init_cache()

In [11]:
class Game_state:
    def __init__(self, game):
        self._game = game
        self._display = show_img()
        self._display.__next__()
        self.high_score = 0
        self.score_array = []
        
    def get_state(self, actions):
#         actions_df.loc[len(actions_df)] = actions[1:]
        state, score, done, high_score = self._game.get()
#         reward = 0.1 # Reward because he is alive
        reward = 0
        
        # Do action
        if actions[0] == 1:
            t = threading.Thread(target=self._game.press_key, args=('q'))
            t.start()
#             self._game.press_key('q')

        elif actions[1] == 1:
            t = threading.Thread(target=self._game.press_key, args=('w'))
            t.start()

        elif actions[2] == 1:
            t = threading.Thread(target=self._game.press_key, args=('o'))
            t.start()
 
        elif actions[3] == 1:
            t = threading.Thread(target=self._game.press_key, args=('p'))
            t.start()
        
        self.score_array.append(score)
        
        if len(self.score_array) > 5:
            self.score_array.pop(0)
            if (self.score_array[0] == self.score_array[1] 
                and self.score_array[1] == self.score_array[2] 
                and self.score_array[2] == self.score_array[3] 
                and self.score_array[3] == self.score_array[4]):
                reward -= 0.1
        
        self._display.send(state)
        
        if done:
            scores_df.loc
            self._game.restart()
            reward += high_score * 10
        
        return state, reward, done

In [12]:
def buildmodel():
    print("Build the model.")
    model = Sequential()
    model.add(Conv2D(32, (5, 5), strides=(3, 3), padding='same',input_shape=(img_cols,img_rows,img_channels), kernel_initializer='he_uniform'))  #300*300*4
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3), strides=(1, 1), padding='same', kernel_initializer='he_uniform'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3), strides=(1, 1), padding='same', kernel_initializer='he_uniform'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(ACTIONS))
    model.compile(loss='mse',optimizer=Adam(lr=LEARNING_RATE))
    
    #create model file if not present
    if not os.path.isfile(loss_file_path):
        model.save_weights('model.h5')
    print("Finish building the model. ")
    
    return model

In [13]:
def trainNetwork(model, game_state, observe=False):
    last_time = time.time()
    # store the previous observations in replay memory
    D = load_obj("D") #load from file system
    # get the first state by doing nothing
    do_nothing = np.zeros(ACTIONS)
    do_nothing[0] = 1
    
    x_t, r_0, terminal = game_state.get_state(do_nothing) # get next step after performing the action
    
    s_t = np.stack((x_t, x_t, x_t, x_t), axis=2) # stack 4 images to create placeholder input
    

    s_t = s_t.reshape(1, s_t.shape[0], s_t.shape[1], s_t.shape[2])  #1*20*40*4
    
    initial_state = s_t 

    if observe :
        OBSERVE = 999999999    #We keep observe, never train
        epsilon = FINAL_EPSILON
        print ("Now we load weight")
        model.load_weights("model.h5")
        adam = Adam(lr=LEARNING_RATE)
        model.compile(loss='mse',optimizer=adam)
        print ("Weight load successfully")    
    else:                       #We go to training mode
        OBSERVE = OBSERVATION
        epsilon = load_obj("epsilon") 
        model.load_weights("model.h5")
        adam = Adam(lr=LEARNING_RATE)
        model.compile(loss='mse',optimizer=adam)

    t = load_obj("time") # resume from the previous time step stored in file system
    while (True): #endless running
        
        loss = 0
        Q_sa = 0
        action_index = 0
        r_t = 0 #reward at 4
        a_t = np.zeros([ACTIONS]) # action at t
        
        #choose an action epsilon greedy
        if t % FRAME_PER_ACTION == 0: #parameter to skip frames for actions
            if  random.random() <= epsilon: #randomly explore an action
                print("----------Random Action----------")
                action_index = random.randrange(ACTIONS)
                a_t[action_index] = 1
            else: # predict the output
                q = model.predict(s_t)       #input a stack of 4 images, get the prediction
                max_Q = np.argmax(q)         # chosing index with maximum q value
                action_index = max_Q 
                a_t[action_index] = 1        # o=> do nothing, 1=> jump
                
        #We reduced the epsilon (exploration parameter) gradually
        if epsilon > FINAL_EPSILON and t > OBSERVE:
            epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / EXPLORE 

        #run the selected action and observed next state and reward
        x_t1, r_t, terminal = game_state.get_state(a_t)
        print('fps: {0}'.format(1 / (time.time()-last_time))) # helpful for measuring frame rate
        last_time = time.time()
        x_t1 = x_t1.reshape(1, x_t1.shape[0], x_t1.shape[1], 1) #1x20x40x1
        s_t1 = np.append(x_t1, s_t[:, :, :, :3], axis=3) # append the new image to input stack and remove the first one
        
        
        # store the transition in D
        D.append((s_t, action_index, r_t, s_t1, terminal))
        if len(D) > REPLAY_MEMORY:
            D.popleft()

        #only train if done observing
        if t > OBSERVE: 
            
            #sample a minibatch to train on
            minibatch = random.sample(D, BATCH)
            inputs = np.zeros((BATCH, s_t.shape[1], s_t.shape[2], s_t.shape[3]))   #32, 20, 40, 4
            targets = np.zeros((inputs.shape[0], ACTIONS))                         #32, 2

            #Now we do the experience replay
            for i in range(0, len(minibatch)):
                state_t = minibatch[i][0]    # 4D stack of images
                action_t = minibatch[i][1]   #This is action index
                reward_t = minibatch[i][2]   #reward at state_t due to action_t
                state_t1 = minibatch[i][3]   #next state
                terminal = minibatch[i][4]   #wheather the agent died or survided due the action
                

                inputs[i:i + 1] = state_t    

                targets[i] = model.predict(state_t)  # predicted q values
                Q_sa = model.predict(state_t1)      #predict q values for next step
                
                if terminal:
                    targets[i, action_t] = reward_t # if terminated, only equals reward
                else:
                    targets[i, action_t] = reward_t + GAMMA * np.max(Q_sa)

            loss += model.train_on_batch(inputs, targets)
            loss_df.loc[len(loss_df)] = loss
            q_values_df.loc[len(q_values_df)] = np.max(Q_sa)
        s_t = initial_state if terminal else s_t1 #reset game to initial frame if terminate
        t = t + 1
        
        # save progress every 1000 iterations
        if t % 1000 == 0:
            print("Now we save model")
            model.save_weights("model.h5", overwrite=True)
            save_obj(D,"D") #saving episodes
            save_obj(t,"time") #caching time steps
            save_obj(epsilon,"epsilon") #cache epsilon to avoid repeated randomness in actions
            loss_df.to_csv("./objects/loss_df.csv",index=False)
            scores_df.to_csv("./objects/scores_df.csv",index=False)
#             actions_df.to_csv("./objects/actions_df.csv",index=False)
            q_values_df.to_csv(q_value_file_path,index=False)
            with open("model.json", "w") as outfile:
                json.dump(model.to_json(), outfile)
            clear_output()
        # print info
        state = ""
        if t <= OBSERVE:
            state = "observe"
        elif t > OBSERVE and t <= OBSERVE + EXPLORE:
            state = "explore"
        else:
            state = "train"

        print("TIMESTEP", t, "/ STATE", state,             "/ EPSILON", epsilon, "/ ACTION", action_index, "/ REWARD", r_t,             "/ Q_MAX " , np.max(Q_sa), "/ Loss ", loss)

    print("Episode finished!")
    print("************************")

In [14]:
#main function
def playGame(observe=False):
    game = qwop.Game()
    game_state = Game_state(game)    
    model = buildmodel()
    try:
        trainNetwork(model,game_state,observe=observe)
    except StopIteration:
        game.end()

In [15]:
playGame(observe=False);

TIMESTEP 54000 / STATE explore / EPSILON 0.04615489899991773 / ACTION 1 / REWARD 10.0 / Q_MAX  129.54584 / Loss  41.823036193847656
fps: 0.027487569411813694
TIMESTEP 54001 / STATE explore / EPSILON 0.04615389999991773 / ACTION 3 / REWARD 0 / Q_MAX  136.53079 / Loss  47.516815185546875
fps: 0.7885982300505595
TIMESTEP 54002 / STATE explore / EPSILON 0.04615290099991773 / ACTION 2 / REWARD 0 / Q_MAX  124.00511 / Loss  27.419479370117188
fps: 0.7412474109471868
TIMESTEP 54003 / STATE explore / EPSILON 0.04615190199991773 / ACTION 2 / REWARD 0 / Q_MAX  142.401 / Loss  55.99506378173828
fps: 0.7727533006370976
TIMESTEP 54004 / STATE explore / EPSILON 0.046150902999917726 / ACTION 1 / REWARD 0 / Q_MAX  124.18278 / Loss  39.7996940612793
fps: 0.7781655955142024
TIMESTEP 54005 / STATE explore / EPSILON 0.046149903999917724 / ACTION 2 / REWARD 0 / Q_MAX  115.12438 / Loss  83.25408935546875
fps: 0.7467833135760494
TIMESTEP 54006 / STATE explore / EPSILON 0.04614890499991772 / ACTION 2 / REWARD 

fps: 0.7961329798764166
TIMESTEP 54053 / STATE explore / EPSILON 0.04610195199991765 / ACTION 0 / REWARD 0 / Q_MAX  143.50638 / Loss  92.31919860839844
fps: 0.7867371064841083
TIMESTEP 54054 / STATE explore / EPSILON 0.04610095299991765 / ACTION 3 / REWARD 0 / Q_MAX  159.25093 / Loss  47.91556167602539
fps: 0.7948672806195498
TIMESTEP 54055 / STATE explore / EPSILON 0.04609995399991765 / ACTION 3 / REWARD 0 / Q_MAX  43.225647 / Loss  41.93206024169922
fps: 0.8051070106224577
TIMESTEP 54056 / STATE explore / EPSILON 0.046098954999917646 / ACTION 1 / REWARD 0 / Q_MAX  48.17327 / Loss  117.47930145263672
fps: 0.8250353525826318
TIMESTEP 54057 / STATE explore / EPSILON 0.046097955999917645 / ACTION 1 / REWARD 0 / Q_MAX  137.92683 / Loss  72.64205932617188
fps: 0.7867371064841083
TIMESTEP 54058 / STATE explore / EPSILON 0.04609695699991764 / ACTION 3 / REWARD 0 / Q_MAX  146.02002 / Loss  58.10223388671875
fps: 0.7680052500358889
TIMESTEP 54059 / STATE explore / EPSILON 0.04609595799991764 /

fps: 0.7848844531348098
TIMESTEP 54106 / STATE explore / EPSILON 0.04604900499991757 / ACTION 2 / REWARD 0 / Q_MAX  123.33811 / Loss  41.42634963989258
fps: 0.8169466950726921
TIMESTEP 54107 / STATE explore / EPSILON 0.04604800599991757 / ACTION 2 / REWARD 0 / Q_MAX  145.239 / Loss  29.370014190673828
fps: 0.8109837211748473
TIMESTEP 54108 / STATE explore / EPSILON 0.04604700699991757 / ACTION 0 / REWARD 0 / Q_MAX  140.36803 / Loss  34.443382263183594
fps: 0.814285500730843
TIMESTEP 54109 / STATE explore / EPSILON 0.046046007999917565 / ACTION 1 / REWARD 0 / Q_MAX  118.76502 / Loss  92.72309875488281
fps: 0.8129616309534123
TIMESTEP 54110 / STATE explore / EPSILON 0.046045008999917564 / ACTION 1 / REWARD 0 / Q_MAX  137.87689 / Loss  40.134578704833984
fps: 0.793605449256167
TIMESTEP 54111 / STATE explore / EPSILON 0.04604400999991756 / ACTION 1 / REWARD 0 / Q_MAX  108.09003 / Loss  24.805252075195312
fps: 0.8012361907094285
TIMESTEP 54112 / STATE explore / EPSILON 0.04604301099991756 /

fps: 0.8129617885258587
TIMESTEP 54160 / STATE explore / EPSILON 0.04599505899991749 / ACTION 3 / REWARD 0 / Q_MAX  149.30211 / Loss  86.38479614257812
fps: 0.7961328287601405
TIMESTEP 54161 / STATE explore / EPSILON 0.045994059999917486 / ACTION 3 / REWARD 0 / Q_MAX  146.39233 / Loss  143.9739227294922
fps: 0.8038126397562904
TIMESTEP 54162 / STATE explore / EPSILON 0.045993060999917484 / ACTION 1 / REWARD 0 / Q_MAX  141.99776 / Loss  134.0864715576172
fps: 0.8136233429736379
TIMESTEP 54163 / STATE explore / EPSILON 0.04599206199991748 / ACTION 0 / REWARD 0 / Q_MAX  76.78285 / Loss  50.317291259765625
fps: 0.7917201960847234
TIMESTEP 54164 / STATE explore / EPSILON 0.04599106299991748 / ACTION 1 / REWARD 0 / Q_MAX  135.70982 / Loss  116.49728393554688
fps: 0.8109837211748473
TIMESTEP 54165 / STATE explore / EPSILON 0.04599006399991748 / ACTION 1 / REWARD 0 / Q_MAX  130.41011 / Loss  63.82291030883789
fps: 0.8129614733810272
TIMESTEP 54166 / STATE explore / EPSILON 0.04598906499991748 

fps: 0.7942357133619053
TIMESTEP 54214 / STATE explore / EPSILON 0.045941112999917405 / ACTION 0 / REWARD 0 / Q_MAX  148.79825 / Loss  123.35508728027344
fps: 0.7910940656590661
TIMESTEP 54215 / STATE explore / EPSILON 0.045940113999917404 / ACTION 1 / REWARD 0 / Q_MAX  163.56215 / Loss  114.16903686523438
fps: 0.8077079875451751
TIMESTEP 54216 / STATE explore / EPSILON 0.0459391149999174 / ACTION 1 / REWARD 0 / Q_MAX  67.611786 / Loss  96.5322494506836
fps: 0.8031669368908586
TIMESTEP 54217 / STATE explore / EPSILON 0.0459381159999174 / ACTION 3 / REWARD 0 / Q_MAX  127.69089 / Loss  77.70310974121094
fps: 0.7936052990978834
TIMESTEP 54218 / STATE explore / EPSILON 0.0459371169999174 / ACTION 1 / REWARD 0 / Q_MAX  124.56328 / Loss  25.59928321838379
fps: 0.8038123316648824
TIMESTEP 54219 / STATE explore / EPSILON 0.0459361179999174 / ACTION 3 / REWARD -0.1 / Q_MAX  131.18636 / Loss  352.692138671875
fps: 0.8123013090195126
TIMESTEP 54220 / STATE explore / EPSILON 0.045935118999917396 /

...

fps: 0.8057558379445455
TIMESTEP 54690 / STATE explore / EPSILON 0.04546558899991668 / ACTION 3 / REWARD 0 / Q_MAX  123.71778 / Loss  35.754493713378906
fps: 0.8038126397562904
TIMESTEP 54691 / STATE explore / EPSILON 0.04546458999991668 / ACTION 1 / REWARD 0 / Q_MAX  39.512413 / Loss  53.94624710083008
fps: 0.8142858169030299
TIMESTEP 54692 / STATE explore / EPSILON 0.045463590999916675 / ACTION 3 / REWARD 0 / Q_MAX  139.51962 / Loss  203.2947235107422
fps: 0.8044593816775695
TIMESTEP 54693 / STATE explore / EPSILON 0.045462591999916674 / ACTION 1 / REWARD 0 / Q_MAX  149.91716 / Loss  62.13154983520508
fps: 0.7942357133619053
TIMESTEP 54694 / STATE explore / EPSILON 0.04546159299991667 / ACTION 1 / REWARD 0 / Q_MAX  147.80908 / Loss  20.038026809692383
----------Random Action----------
fps: 0.7818160602532572
TIMESTEP 54695 / STATE explore / EPSILON 0.04546059399991667 / ACTION 0 / REWARD 0 / Q_MAX  133.02795 / Loss  135.30047607421875
fps: 0.8038126397562904
TIMESTEP 54696 / STATE ex

fps: 0.7581069449781496
TIMESTEP 54743 / STATE explore / EPSILON 0.0454126419999166 / ACTION 2 / REWARD 0 / Q_MAX  136.68216 / Loss  98.01881408691406
fps: 0.7495822552391002
TIMESTEP 54744 / STATE explore / EPSILON 0.045411642999916596 / ACTION 2 / REWARD 0 / Q_MAX  8.401579 / Loss  61.450279235839844
fps: 0.7942357133619053
TIMESTEP 54745 / STATE explore / EPSILON 0.045410643999916595 / ACTION 3 / REWARD 0 / Q_MAX  134.95001 / Loss  57.80274200439453
fps: 0.7830405047538166
TIMESTEP 54746 / STATE explore / EPSILON 0.04540964499991659 / ACTION 1 / REWARD 0 / Q_MAX  5.956073 / Loss  66.1451416015625
fps: 0.7818160602532572
TIMESTEP 54747 / STATE explore / EPSILON 0.04540864599991659 / ACTION 1 / REWARD 0 / Q_MAX  139.91927 / Loss  125.8819351196289
error:  could not convert string to float: '\\... Me' 
score:  \... Me
fps: 0.7183497308096434
TIMESTEP 54748 / STATE explore / EPSILON 0.04540764699991659 / ACTION 1 / REWARD 2.0 / Q_MAX  80.58055 / Loss  54.55659866333008
error:  could not

fps: 0.8044592273838481
TIMESTEP 54796 / STATE explore / EPSILON 0.04535969499991652 / ACTION 0 / REWARD 0 / Q_MAX  113.665344 / Loss  56.67031478881836
fps: 0.8077082986301344
TIMESTEP 54797 / STATE explore / EPSILON 0.045358695999916515 / ACTION 1 / REWARD 0 / Q_MAX  134.6278 / Loss  45.565616607666016
fps: 0.8012361907094285
TIMESTEP 54798 / STATE explore / EPSILON 0.045357696999916514 / ACTION 3 / REWARD 0 / Q_MAX  140.19362 / Loss  26.787315368652344
fps: 0.7745490619291485
TIMESTEP 54799 / STATE explore / EPSILON 0.04535669799991651 / ACTION 1 / REWARD 0 / Q_MAX  155.41223 / Loss  280.5610046386719
fps: 0.7954994004789341
TIMESTEP 54800 / STATE explore / EPSILON 0.04535569899991651 / ACTION 1 / REWARD 0 / Q_MAX  157.18658 / Loss  363.8948974609375
fps: 0.7848844531348098
TIMESTEP 54801 / STATE explore / EPSILON 0.04535469999991651 / ACTION 0 / REWARD 0 / Q_MAX  17.8597 / Loss  54.37299346923828
fps: 0.8136230273156945
TIMESTEP 54802 / STATE explore / EPSILON 0.04535370099991651 /

TIMESTEP 54848 / STATE explore / EPSILON 0.04530774699991644 / ACTION 0 / REWARD 0 / Q_MAX  120.71946 / Loss  141.75942993164062
fps: 0.7948672806195498
TIMESTEP 54849 / STATE explore / EPSILON 0.045306747999916436 / ACTION 0 / REWARD 0 / Q_MAX  117.02642 / Loss  100.18280029296875
fps: 0.8025224241308694
TIMESTEP 54850 / STATE explore / EPSILON 0.045305748999916434 / ACTION 0 / REWARD 0 / Q_MAX  154.07104 / Loss  100.4289321899414
fps: 0.7961329798764166
TIMESTEP 54851 / STATE explore / EPSILON 0.04530474999991643 / ACTION 0 / REWARD 0 / Q_MAX  134.84596 / Loss  72.66163635253906
fps: 0.7368774087547832
TIMESTEP 54852 / STATE explore / EPSILON 0.04530375099991643 / ACTION 3 / REWARD 13.0 / Q_MAX  144.68181 / Loss  171.624267578125
fps: 0.7986763574847822
TIMESTEP 54853 / STATE explore / EPSILON 0.04530275199991643 / ACTION 0 / REWARD 0 / Q_MAX  166.95311 / Loss  37.17001724243164
fps: 0.7541045738143671
TIMESTEP 54854 / STATE explore / EPSILON 0.04530175299991643 / ACTION 1 / REWARD 0

fps: 0.781816205983387
TIMESTEP 54901 / STATE explore / EPSILON 0.045254799999916356 / ACTION 3 / REWARD 0 / Q_MAX  135.09564 / Loss  94.45152282714844
fps: 0.8162797273893377
TIMESTEP 54902 / STATE explore / EPSILON 0.045253800999916355 / ACTION 1 / REWARD 0 / Q_MAX  16.623724 / Loss  66.90069580078125
fps: 0.7942357133619053
TIMESTEP 54903 / STATE explore / EPSILON 0.04525280199991635 / ACTION 3 / REWARD 0 / Q_MAX  176.1976 / Loss  114.14372253417969
fps: 0.8012361907094285
TIMESTEP 54904 / STATE explore / EPSILON 0.04525180299991635 / ACTION 1 / REWARD -0.1 / Q_MAX  164.48697 / Loss  172.69964599609375
fps: 0.7954992496030853
TIMESTEP 54905 / STATE explore / EPSILON 0.04525080399991635 / ACTION 1 / REWARD -0.1 / Q_MAX  167.14023 / Loss  47.32605743408203
fps: 0.8018787916329229
TIMESTEP 54906 / STATE explore / EPSILON 0.04524980499991635 / ACTION 3 / REWARD -0.1 / Q_MAX  149.58644 / Loss  51.6119270324707
fps: 0.7751494096613504
TIMESTEP 54907 / STATE explore / EPSILON 0.04524880599